# Processing Science Images
## <font color=purple> Scaling and Combining
<font color=purple> Written by Maria Funcich    

### <font color=purple> $\star$ Importing necessary libraries

In [1]:
import numpy as np
import astropy
import ccdproc
from ccdproc import CCDData, combiner
from astropy import units as u
import matplotlib.pyplot as plt 
from matplotlib.colors import LogNorm 
from photutils.centroids import centroid_2dg 
from scipy.ndimage import shift 
from photutils import CircularAperture
from photutils import aperture_photometry
import os

### <font color=purple> $\star$ Collecting and Loading the Shifted Science images

In [2]:
# function 'image_collect' that collects all the images from a specified band // reads .fit in adu
## 2 arguments --> directory: location of the .fit
##             --> band: type of filter

def image_collect(directory, band):
    cwd = os.getcwd()                # saving current directory
    os.chdir(directory)              # changing directory
    images = ccdproc.ImageFileCollection(".", glob_include = band)                 # collects all specified band images in the directory
     
    scim = [CCDData.read(fn, unit = "adu") for fn in images.files_filtered()]      # defines "scim" to read the CCD data in adu
    os.chdir(cwd)                    # returning to the defined current directory
    
    return scim                      # returns (value) array

### <font color=purple> $\star$ Aperture Photometry

In [3]:
# function 'photometry' does photometry aperture on the specified/bright stars // prints out the photometry (flux values) table
## 2 arguments --> star_position: (x,y) coordinates of bright stars
##             --> radius: radius of circular aperture

def photometry(star_position, radius): 
    
    apertures = CircularAperture(star_position, r = radius)       # apertures defined by a circle with radius 'r' pixels that will surround the star
    phot_table = aperture_photometry(scim[0], apertures)          # defining a photometry table that calculates and displays the flux for each star in the first image
    print("\n",
          
          "PHOTOMETRY TABLE FOR FIRST IMAGE",
          
          "\n",
          
          phot_table,
         
         "\n")
    
    phot_table=[]       # creates a new array for all our science image photometry tables
    
    print("\n",
          
          "PHOTOMETRY TABLE FOR ALL IMAGES")
    
    for idx, thisimage in enumerate(scim):                                # for loop that runs through all shifted science images
        phot_table.extend([aperture_photometry(thisimage, apertures)])    # calculates the flux for each star in all science image
        print("\n", 
              
              idx, phot_table[idx], 
              
              "\n")
    
    return phot_table
        
    # printing out flux values in adu for each star in first image 
    print("\n",
          
          "APERTURE SUM OF STARS (Flux Values) FOR FIRST IMAGE",
          
          "\n",
          
          phot_table[0]['aperture_sum'])
    

### <font color=purple> $\star$ Positions of bright stars for B, V, R and I band
<font color=purple> 
- Manually picked in ds9 
 - I picked the stars that would be great for aperture photometry (ones that had a defined circular appearance)
 - Chosen from the first science image
    

In [4]:
# bright stars gathered into an array
## coordinates of stars in the first science image 

position_MAR09_V = [(563.394,321.438), (522.479,299.957), (685.053,46.082), (259.311, 50.565), (185.395,39.328), (408.040,430.484), (415.553, 98.852), (30.863,217.533), (124.637,370.334), (517.631,174.968), (593.371,221.861)]
position_MAR09_B = [(522.664,300.336), (563.243, 321.768), (185.549, 39.929), (441.632, 163.022), (429.312, 327.290), (407.941, 431.066), (429.162,327.251), (226.350,77.914), (239.797,135.189), (259.315,50.801)]  
position_MAR09_R = [(563.794,321.598), (408.146,430.232), (294.930,56.057), (259.531,49.832), (30.926,217.501), (124.916,370.236), (408.212,430.140), (522.99,299.781), (49.627,27.463), (329.471,416.622), (415.754,98.519)]
position_MAR09_I = [(592.363,221.662), (48.246,26.965), (293.889,55.849), (258.523,49.820), (607.125,86.216), (592.409,221.516), (406.849,430.163)]

position_MAR29_V = [(602.816,456.921), (354.656,494.391), (381.877,66.898), (304.334,77.417), (664.473,127.188)]
position_MAR29_R = [(183.279,408.884), (106.253,449.351), (381.617,66.790), (227.294,174.99), (106.054,449.626)]
position_MAR29_I = [(183.558,408.779), (197.279,273.663), (602.400,456.617), (354.483,494.117)]


<font color=purple> 
- The shifted images for the R and I filter are extremely saturated, therefore the flux is most likely going to be an overestimate for those images
- Since I-band stars were saturated, many of the stars nearing the middle of the cluster were merged, therefore must pick more 'isolated' stars to avoid including other stars during aperture photometry 

### <font color=purple> $\star$ Photometry Tables & Cross checking fluxes using best ('most photometric') image

<font color=purple> Cross check by calculating the flux ratio of the first star and the best image
- Where best image also known as our reference image is the best photometric one
 - Stars are well round and well defined 
 - Clear (no blurring)

In [5]:
# function 'cross_check' calculates the flux ratio // reference image over the all science images
## 1 argument --> best image: our reference image

def cross_check(best_image):
    print("\n",
         
         "CROSS CHECKING"
         
         "\n")
    
    for idx, thisimage in enumerate(scim):      # running through all science images
        print("\n",
              
              "Flux ratio of Best Image over Image ",
              
              idx,
              
              "\n")                              # prints the index for each image (0 is the first image in this case)
        
        print(phot_table[best_image]['aperture_sum']/phot_table[idx]['aperture_sum'])                # we use our "best" image as our reference image 
        
        print("\n",
              
              "Median value of Flux across the image:",
              
              "\n",
              
              np.ma.median(phot_table[best_image]['aperture_sum']/phot_table[idx]['aperture_sum']))  # calculates the median value of the flux for each image  
        

In [6]:
# function 'photometry' does photometry aperture on the specified/bright stars // prints out the photometry (flux values) table
## 4 arguments --> directory: location of the .fit
##             --> best_image: reference image
##             --> band: type of filter
##             --> filename: name of final combined image

def scale_and_combine(directory, best_image, band, filename):
    # just changing the directories...
    cwd = os.getcwd()
    os.chdir(directory)
    
    # collecting images...
    images = ccdproc.ImageFileCollection(".",glob_include = band)
    scim = [CCDData.read(fn) for fn in images.files_filtered()]

    ## scaling
    # for loop calculates median flux ratio for each image and displays them
    for idx, thisimage in enumerate(scim): 
        m = np.ma.median(phot_table[best_image]['aperture_sum'] / phot_table[idx]['aperture_sum'])
        print("Image ", idx, ": ", m)                                       # scaling values
        scim[idx].data = scim[idx].multiply(m * u.adu).data      # implement the scaling (multiplying all science images with the scaled value 'm')

    ## median combine
    #calculates median for the images and combines them
    sci_median = ccdproc.combine(scim, method = 'median',dtype = np.float32, 
                                     minmax_clip = True, minmax_clip_min = -500)
    
    # Save the header information, like exposure time and airmass
    # of best image (Evgenii). This will be needed later for calculating magnitudes
    sci_median.header = scim[best_image].header
        
    os.chdir(cwd)                     #  return to defined directory 
   
    sci_median.write(filename)       # writes the median combined image as a .fit
    
    return sci_median
        
    del(scim)

In [7]:
def median_statistics(star_position, radius, best_image):
    apertures = CircularAperture(star_position, r = radius)      
    print('Median combine',
         
         "\n")     
    
    phot_table_median = aperture_photometry(sci_median, apertures)    # calculates and displays flux count for each star       
    print("\n", 'Median pixel value', np.ma.median(sci_median)) # calculates median pixel value in ADU
    print('Standard deviation', np.std(sci_median[400:500,400:500]))  # calculating the standard devitation between 400 to 500 pixels
    
    print("\n",
          
          "PHOTOMETRY TABLE FOR MEDIAN COMBINE IMAGE",
          
          "\n",
          
          phot_table_median,
         
         "\n")      
    
    print("\n",
          
          "Cross Checking: flux ratio with best image",
          
          "\n",
          
        phot_table[best_image]['aperture_sum']/phot_table_median['aperture_sum'],
         
         "\n")   
    
    print("\n",
          
          "Median value of the flux ratio: ",
          
        np.ma.median(phot_table[best_image]['aperture_sum']/phot_table_median['aperture_sum']))  # median value for the flux ratio

### <font color=purple> $\star$ Scaling and Combining each band

### <font color=green> --------------------------------------------------------------------------B-Band---------------------------------------------------------------------------

<font color=purple> **Date:** March 9th | **Exposure Time:** 60.0s

In [8]:
scim = image_collect("../040_shift/data/shifted/march_09_2018","NGC_3201_B*")

phot_table = photometry(position_MAR09_B, 10)

cross_check(6)

sci_median = scale_and_combine("../040_shift/data/shifted/march_09_2018",6,"NGC_3201_B*","NGC_3201_B_median_60.0s.fits")

median_statistics(position_MAR09_B, 10, 6)

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]

 PHOTOMETRY TABLE FOR FIRST IMAGE 
  id xcenter ycenter    aperture_sum   
      pix     pix          adu        
--- ------- ------- ------------------
  1 522.664 300.336 

Image  0 :  0.974155776917841
Image  1 :  1.001795468538711
Image  2 :  0.9916708707200723
Image  3 :  1.0077267494115212
Image  4 :  1.0058340890193707
Image  5 :  1.005324902334501
Image  6 :  1.0
Image  7 :  0.9968152727274724
Image  8 :  1.0171820081382363
Image  9 :  0.9997202523512361
Image  10 :  0.9745198122888536
Image  11 :  0.9927735595030572
Image  12 :  0.9858997006869752
Image  13 :  0.9985021343574352
Image  14 :  0.9807772327308334
Image  15 :  1.0019858671134192
Median combine 


 Median pixel value -0.23956571519374847
Standard deviation 10.668674

 PHOTOMETRY TABLE FOR MEDIAN COMBINE IMAGE 
  id xcenter ycenter    aperture_sum   
      pix     pix          adu        
--- ------- ------- ------------------
  1 522.664 300.336  6149.809356616783
  2 563.243 321.768  4804.110411791312
  3 185.549  39.929 11920.413684534538
  4 441.632 163.022  8865.445135136337
  5 429.312  327.29  6784.203067850764
  6 407.941 431.066 6206.8319032394575
  7 429.162 327.251  6783.85033

<font color=purple>
    
Photometry table for images:
- using a radius of 10 due to the stars having an average value radius below this
- The flux values in ADU for the stars are all consistent across the images

Cross checking:
- Using image 7 as the reference/best image
- An image with a greater flux than our reference image will give us a value less than 1, whereas if it was smaller it would be greater than 1.
- scalar values are all close to 1, this is a good sign

- Median value of the flux ratio:  1.0221964828724026 1 / adu <br>
    
Happy with stack!


### <font color=green> --------------------------------------------------------------------------V-Band---------------------------------------------------------------------------

<font color=purple> **Date:** March 9th | **Exposure Time:** 60.0s

In [9]:
scim = image_collect("../040_shift/data/shifted/march_09_2018","NGC_3201_V*")

phot_table = photometry(position_MAR09_V, 10)

cross_check(12)

sci_median = scale_and_combine("../040_shift/data/shifted/march_09_2018",12,"NGC_3201_V*","NGC_3201_V_median_60.0s.fits")

median_statistics(position_MAR09_V, 10, 12)

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]

 PHOTOMETRY TABLE FOR FIRST IMAGE 
  id xcenter ycenter    aperture_sum   
      pix     pix          adu        
--- ------- ------- ------------------
  1 563.394 321.438 

Image  0 :  0.9828006665546155
Image  1 :  1.0005169620484855
Image  2 :  1.007002859931835
Image  3 :  1.0122899507973777
Image  4 :  1.0126433399980965
Image  5 :  1.0078866492584748
Image  6 :  1.010488761792304
Image  7 :  1.0106417248925794
Image  8 :  0.9921355123936455
Image  9 :  1.0001494738707044
Image  10 :  1.001057109165159
Image  11 :  1.005441207619928
Image  12 :  1.0
Image  13 :  0.994072586455405
Image  14 :  1.0033705652686808
Image  15 :  1.0017601008047692
Median combine 


 Median pixel value -0.809073805809021
Standard deviation 50.465797

 PHOTOMETRY TABLE FOR MEDIAN COMBINE IMAGE 
  id xcenter ycenter    aperture_sum   
      pix     pix          adu        
--- ------- ------- ------------------
  1 563.394 321.438 22369.942581064468
  2 522.479 299.957 29233.376882996585
  3 685.053  46.082  4778.992021267439
  4 259.311  50.565 16095.178838641115
  5 185.395  39.328   67771.7319781458
  6  408.04 430.484 30775.741560882816
  7 415.553  98.852  15415.26776437

<font color=purple>
    
Photometry table for images:
- using a radius of 10 due to the stars having an average value radius below this
- The flux values in ADU for the stars are all consistent across the images

Cross checking:
- Using image 13 as the reference/best image
- scalar values are all close to 1, this is a good sign

- Median value of the flux ratio:  1.0152996410680588 1 / adu <br>


<font color=purple> **Date:** March 29th | **Exposure Time:** 30.0s

In [10]:
scim = image_collect("../040_shift/data/shifted/march_29_2018","NGC_3201_V_30*")

phot_table = photometry(position_MAR29_V, 10)

cross_check(0)

sci_median = scale_and_combine("../040_shift/data/shifted/march_29_2018",0,"NGC_3201_V_30*","NGC_3201_V_median_30.0s.fits")

median_statistics(position_MAR29_V, 10, 0)

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]

 PHOTOMETRY TABLE FOR FIRST IMAGE 
  id xcenter ycenter    aperture_sum   
      pix     pix          adu        
--- ------- ------- ------------------
  1 602.816 456.921 32726.209454728458
  2 354.656 494.391  16661.72157043775
  3 381.877  66.898 14352.402735079297
  4 304.334  77.417  5934.319585238452
  5 664.473 127.188  4759.432115327679 


 PHOTOMETRY TABLE FOR ALL IMAGES

 0  id xcenter ycenter    aperture_sum   
      pix     pix          adu        
--- ------- ------- ------------------
  1 602.816 456.921 32726.209454728458
  2 354.656 494.391  16661.72157043775
  3 381.877  66.898 14352.402735079297
  4 304.334  77.417  5934.319585238452
  5 664.473 127.188  4759.432115327679 


 1  id xcenter ycenter    aperture_sum   
      pix     pix 

<font color=purple>
Only has 2 images to stack | Photometric night <br>
    
Photometry table for images:
- using a radius of 10 due to the stars having an average value radius below this
- The flux values in ADU for the stars are all consistent across the images

Cross checking:
- Using image 1 as the reference/best image
- scalar values are all close to 1, this is a good sign

- Median value of the flux ratio:  0.9999999982621381 1 / adu <br>


### <font color=green> --------------------------------------------------------------------------R-Band---------------------------------------------------------------------------

<font color=purple> **Date:** March 9th | **Exposure Time:** 60.0s

In [ ]:
scim = image_collect("../040_shift/data/shifted/march_09_2018","NGC_3201_R*")

phot_table = photometry(position_MAR09_R, 12)

cross_check(8)

sci_median = scale_and_combine("../040_shift/data/shifted/march_09_2018",8,"NGC_3201_R*","NGC_3201_R_median_60.0s.fits")

median_statistics(position_MAR09_R, 12, 8)

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]

 PHOTOMETRY TABLE FOR FIRST IMAGE 
  id xcenter ycenter

<font color=purple> **Date:** March 29th | **Exposure Time:** 30.0s

In [ ]:
scim = image_collect("../040_shift/data/shifted/march_29_2018","NGC_3201_R_30*")

phot_table = photometry(position_MAR29_R, 12)

cross_check(0)

sci_median = scale_and_combine("../040_shift/data/shifted/march_29_2018",0,"NGC_3201_R_30*","NGC_3201_R_median_30.0s.fits")

median_statistics(position_MAR29_R, 12, 0)

### <font color=green> --------------------------------------------------------------------------I-Band---------------------------------------------------------------------------

<font color=purple> **Date:** March 9th | **Exposure Time:** 60.0s

In [ ]:
scim = image_collect("../040_shift/data/shifted/march_09_2018","NGC_3201_I*")
phot_table = photometry(position_MAR09_I, 12)
cross_check(9)

sci_median = scale_and_combine("../040_shift/data/shifted/march_09_2018",9,"NGC_3201_I*","NGC_3201_I_median_60.0s.fits")

median_statistics(position_MAR09_I, 12, 9)

<font color=purple> **Date:** March 29th | **Exposure Time:** 30.0s

In [ ]:
scim = image_collect("../040_shift/data/shifted/march_29_2018","NGC_3201_I_30*")

phot_table = photometry(position_MAR29_I, 12)

cross_check(0)

sci_median = scale_and_combine("../040_shift/data/shifted/march_29_2018",0,"NGC_3201_I_30*","NGC_3201_I_median_30.0s.fits")

median_statistics(position_MAR29_I, 12, 0)

Best to use Median and not an average combine of an image for this case 

## Copy B photometric image (Evgenii)

Since we have only a single B-band photometric image, we don't scale/stack it. But I still want all images after this stage be located in the same directory. So I just copy the B image to destination directory.

In [ ]:
from shutil import copyfile

copyfile(src="../040_shift/data/shifted/march_29_2018/NGC_3201_B_30.000secs_00000472.fit",
         dst="../050_scaling_and_combining/march_29_2018_stacked/NGC_3201_B_median_30.0s.fits")

### <font color=purple> $\star$ Displaying and Checking the final scaled and median stacked images

In [ ]:
def display(directory, file, vmax_, vmin_, title):              
    cwd = os.getcwd()
    os.chdir(directory)
    image = CCDData.read(file, unit="adu")
    fig, ax = plt.subplots(figsize = (12,7))

    plt.rcParams.update({'font.size':14 })
    plt.imshow(image, cmap='gray', vmax=vmax_, vmin=vmin_)
    plt.xlabel('x-pixels')
    plt.ylabel('y-pixels')
    plt.title(title)
    plt.colorbar()
    os.chdir(cwd)

In [ ]:
display("march_09_2018_stacked", "NGC_3201_B_median_60.0s.fits", 14, -9, "NGC 3201 B-Band 60.0s")

<font color=purple> Displaying the images using ds9 instead of mayplotlib since images are scaled down and blurry... 
    
- In terminal: "ds9 -zscale NGC_3201*"
    


### <font color=purple> $\star$ NOTES:
    
<font color=purple> 
    
- The 30.0 second exposures from the photometric night (29th of March) only had 2 images to stack (except the B-band which only had 1 - no need to stack...)
    - Out of all the bands, each set always had 1 blurry image <br>
    
    
- The I-band and R-band both have indistguishable stars nearing the centre of the cluster, this is because a lot of the stars ended up merging after stacking...This occured due to the over saturation of the raw images and noticeable shifts. <br>
    
    
- You can notice that each band picked up a lot of faint stars, the noise has also decreased (number of counts per pixel decreases across the background of the image) <br>
    
    
- Best stacked images were from the B and V bands 